## HW1
이미지 파일에

1) zero padding을 크기 1만큼 적용한 뒤 (즉 위 아래 양 옆으로 0을 한겹만)

2) stride를 (1,1)로 filter A를 적용하고

3) stride (2,2)로 2 * 2 maxpooling을 적용했을 때의 이미지 파일은 어떻게 생겼는지 예측하기


In [1]:
import numpy as np

In [2]:
image= np.array([[2,0,2,1,3,2],[0,2,0,2,2,2],[1,0,1,3,1,1],[0,0,1,1,1,0],[0,1,3,4,1,0],[0,1,0,0,5,2]])
image

array([[2, 0, 2, 1, 3, 2],
       [0, 2, 0, 2, 2, 2],
       [1, 0, 1, 3, 1, 1],
       [0, 0, 1, 1, 1, 0],
       [0, 1, 3, 4, 1, 0],
       [0, 1, 0, 0, 5, 2]])

1) zero padding을 크기 1만큼 적용하기.

https://m.blog.naver.com/wideeyed/221665256911 참고, np.pad함수 이용

In [3]:
padding= np.pad(image, ((1,1),(1,1)), 'constant', constant_values=0)

2) stride를 (1,1)로 filter A를 적용하고

In [4]:
filter_A = np.identity(3, dtype=int)
filter_A

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [5]:
row_num= padding.shape[0]- filter_A.shape[0]+1
col_num= padding.shape[1]- filter_A.shape[1]+1

#output image= (input size - filter size+ 2* padding)/stride +1
#여기서는 (6-3+2*1)/1+1=6 즉 6*6사이즈의 output

In [6]:
result2 = []

for i in range(row_num) :
    for j in range(col_num):
        result=np.sum(padding[i:i+filter_A.shape[0], j:j+ filter_A.shape[1]] * filter_A)
        result2=np.append(result2, result)
        
result3=np.array(result2,dtype=int).reshape(row_num,col_num)
print(result3)

[[ 4  0  4  3  5  2]
 [ 0  5  3  5  4  5]
 [ 1  1  4  4  3  3]
 [ 1  4  5  3  4  1]
 [ 1  1  3 10  4  1]
 [ 0  1  1  3  9  3]]


3) stride (2,2)로 2 * 2 maxpooling을 적용했을 때의 이미지 파일은 어떻게 생겼는지 예측하기

In [7]:
stride=2
filter_size=2

In [8]:
print(result3)

[[ 4  0  4  3  5  2]
 [ 0  5  3  5  4  5]
 [ 1  1  4  4  3  3]
 [ 1  4  5  3  4  1]
 [ 1  1  3 10  4  1]
 [ 0  1  1  3  9  3]]


In [9]:
result5=[]

for i in range(0,result3.shape[0]-1, stride):
    for j in range(0,result3.shape[1]-1, stride):
        result4= np.max(result3[i:i+stride, j:j+stride])
        result5=np.append(result5, result4)
        
result6=np.array(result5,dtype=int).reshape(3,3)       
print(result6)

#output image= (input size - filter size+ 2* padding)/stride +1
#여기서는 (6-2+ 2*0)/2+1=3 즉 3*3사이즈의 output

[[ 5  5  5]
 [ 4  5  4]
 [ 1 10  9]]


## HW2

1) layer1의 kernel size를 4로 늘리고, layer 3의 stride를 2로 늘려라

2) layer3의 output size를 구하라

3) cost와 accuracy가 기존 cnn 모델에 비해 어떻게 변할 것인가?

4) 코딩하여 cost와 accuracy 확인해보기

In [10]:
#1) 라이브러리 가져오기
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [11]:
#2) GPU 사용설정+ random value 위한 seed 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [12]:
#3) 학습에 사용되는 parameters설정
learning_rate=0.001
training_epochs=14
batch_size=100

In [13]:
#4) 데이터셋 가져오기
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [14]:
#4-1) 학습에 쓰기 편하게 loader 만들기
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size= batch_size,
                                         shuffle=True,
                                         drop_last=True)
#shuffle=True: 무작위 순서로 batch를 불러올지.
#drop_last: batch_size로 자를 때 맨 마지막에 남는 데이터를 사용할 것인가 버릴 것인가, True면 버리는 것!

1) layer1의 kernel size를 4로 늘리고, layer 3의 stride를 2로 늘려라

2) layer3의 output size를 구하라

3) cost와 accuracy가 기존 cnn 모델에 비해 어떻게 변할 것인가?

4) 코딩하여 cost와 accuracy 확인해보기

In [15]:
#CNN Model
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size = 4, stride = 1, padding = 1), 
            # kernel_size = 4, filter의 사이즈 정하기 
            #1) layer1의 kernel size를 4로 늘리고, 
            #image in ?
            #Conv; ? x  27 x 27 (;= (28-4+2*1)/1+1) x 32 
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
            #pool;  ?  x 13 x 13 (;= (27-2+2*0)/2+1) x 32
            
        
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            #Conv; ? x 13 x 13 (;=(13-3+2*1)/1+1)x 64 
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
            #pool; ? x 6 x 6 (;=(13-2+2*0)/2+1)x 64 


        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 2, padding = 1),
            #1) layer 3의 stride를 2로 늘려라
            #conv; ?  x 3x 3 (;=(6-3+2*1)/2+1) x 128
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
            #pool; ? x 2 x 2 (;=(3-2+2*0)/2+1)x 128 
        
        self.fc1 = torch.nn.Linear(128 * 2 * 2, 625, bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p = 1 - self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out
        

In [16]:
#intantiate CNN modle
model= CNN().to(device)

In [17]:
#6) loss fuctuion (criterion)을 선택하고 최적화 도구 선택(optimizer)
criterion= torch.nn.CrossEntropyLoss().to(device) #Softmax is initially computed
optimizer= torch.optim.Adam(model.parameters(),lr=learning_rate)

In [18]:
#7) 모델 학습 및 loss check (criterion의 output)
#linear와 달리 data를 reshape할 필요가 없기 때문에 X.view(-1,28*28).to(device)에서 X.to(device로 변경)

total_batch = len(data_loader)

print('Learning started. It takes sometime')

for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device) # 요기!
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished')

Learning started. It takes sometime
[Epoch:    1] cost = 0.260818541
[Epoch:    2] cost = 0.0653349981
[Epoch:    3] cost = 0.0462695546
[Epoch:    4] cost = 0.0356797874
[Epoch:    5] cost = 0.0290069245
[Epoch:    6] cost = 0.0246454626
[Epoch:    7] cost = 0.0211415365
[Epoch:    8] cost = 0.0169709064
[Epoch:    9] cost = 0.0155284861
[Epoch:   10] cost = 0.0168496426
[Epoch:   11] cost = 0.0121403234
[Epoch:   12] cost = 0.0110005122
[Epoch:   13] cost = 0.0121472739
[Epoch:   14] cost = 0.00828384608
Learning Finished


In [19]:
#8) 학습된 모델의 성능 확인
## test data의 x변수를 읽을 때 구조가 (data 수, channel 수 , 가로, 세로) 의 순서로 변화
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test) 
    correct_prediction = torch.argmax(prediction,1)== Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy:  0.9837999939918518


#### 2) layer3의 output size를 구하라

2 * 2 * 128




#### 3) cost와 accuracy가 기존 cnn 모델에 비해 어떻게 변할 것인가?

#### 4)코딩하여 cost와 accuracy 확인해보기

In [20]:
#기존의 CNN 모델과의 비교를 위해 기존 CNN 모델 돌리기
#기존 CNN Model

# CNN model 

class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
        
    
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 1))
            
        self.fc1 = torch.nn.Linear(4*4*128, 625 ,bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p = 1 - self.keep_prob))
            
        self.fc2 = torch.nn.Linear(625, 10 ,bias = True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [22]:
model = CNN().to(device)

In [23]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [24]:
#모델 학습
total_batch = len(data_loader)

print('Learning started. It takes sometime')

for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device) # 요기!
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished')

Learning started. It takes sometime
[Epoch:    1] cost = 0.187037423
[Epoch:    2] cost = 0.051764328
[Epoch:    3] cost = 0.0369065069
[Epoch:    4] cost = 0.0284132678
[Epoch:    5] cost = 0.023815278
[Epoch:    6] cost = 0.020001471
[Epoch:    7] cost = 0.0167973842
[Epoch:    8] cost = 0.0149747962
[Epoch:    9] cost = 0.0123901125
[Epoch:   10] cost = 0.0110296924
[Epoch:   11] cost = 0.0103201037
[Epoch:   12] cost = 0.0100688282
[Epoch:   13] cost = 0.00859705824
[Epoch:   14] cost = 0.00773246307
Learning Finished


In [25]:
#모델 성능 확인
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test) 
    correct_prediction = torch.argmax(prediction,1)== Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

Accuracy:  0.9542999863624573


cost는 기존 모델에 비해 높게 나오고, accuracy 또한 기존 모델에 비해 높게 나왔다. 그런데 거의 차이가 없는 듯 하다...

In [26]:
class CNN(torch.nn.Module):
    
    def __init__(self):
        
        super(CNN,self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(torch.nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=2,stride=2, padding=1))
        
        self.fc1 = torch.nn.Linear(4*4*128,625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(self.fc1, torch.nn.ReLU(), torch.nn.Dropout(p=1-self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
   
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [27]:
model = CNN().to(device)

In [28]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [29]:
#모델 학습
total_batch = len(data_loader)

print('Learning started. It takes sometime')

for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device) # 요기!
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished')

Learning started. It takes sometime
[Epoch:    1] cost = 0.175449371
[Epoch:    2] cost = 0.0497100502
[Epoch:    3] cost = 0.0375072099
[Epoch:    4] cost = 0.0281976983
[Epoch:    5] cost = 0.0210385974
[Epoch:    6] cost = 0.0204906464
[Epoch:    7] cost = 0.0174233206
[Epoch:    8] cost = 0.0139079904
[Epoch:    9] cost = 0.0118833585
[Epoch:   10] cost = 0.0117938006
[Epoch:   11] cost = 0.0100219361
[Epoch:   12] cost = 0.00921284873
[Epoch:   13] cost = 0.00853138231
[Epoch:   14] cost = 0.00713493442
Learning Finished


In [30]:
#모델 성능 확인
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test) 
    correct_prediction = torch.argmax(prediction,1)== Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

Accuracy:  0.9858999848365784
